In [61]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from prophet.plot import plot_plotly,plot_components_plotly
from prophet import Prophet

In [43]:
def calc_index(given_lat,given_lon):
    lon_index = int(((np.floor(given_lon/0.625) * 0.625) - (-180))/0.625)
    lon_index = 0 if lon_index == 576 else lon_index
    lat_index = int(((np.floor(given_lat/0.5) * 0.5) - (-90))/0.5)
    return lat_index,lon_index

In [44]:
def get_conc(folder_name,lat_index,lon_index):
    conc_column = np.empty((0,),dtype='float32')
    data_folder = os.listdir(f'../data/{folder_name}')
    for data_file in data_folder:
        data = xr.open_dataset(f"../data/{folder_name}/{data_file}")
        if folder_name == 'aqi1':
            code = 'DUSMASS'
        elif folder_name == 'aqi2':
            code = 'COSC'
        elif folder_name == 'aqi3':
            code = 'TO3'
        else:
            code = 'Invalid'
        conc_data = np.array(data[code].values)[0]
        conc_val = conc_data[lat_index][lon_index]
        conc_column = np.append(conc_column,conc_val)
    return conc_column

In [45]:
date_column = pd.date_range(start='1/1/2005',end='12/1/2023',freq='MS')

In [46]:
given_lat= -20.296059
given_lon= -85.824539
lat_index,lon_index = calc_index(given_lat,given_lon)

In [47]:
pm_conc = get_conc('aqi1',lat_index,lon_index)
co_conc = get_conc('aqi2',lat_index,lon_index)
o3_conc = get_conc('aqi3',lat_index,lon_index)

In [48]:
pm_conc = pm_conc * 1e9            # 1 kg/m^3 = 1e9 ug/m^3
co_conc = co_conc * 1.15 * 1e-3    # 1ppb = 1.15 * 1e-3 mg/m^3 for CO
o3_conc = o3_conc * 0.1            # 10 percent of total ozone column

In [54]:
def calc_aqi(pm_conc,co_conc,o3_conc):
    min_aqi = [0,51,101,201,301,401]
    max_aqi = [50,100,200,300,400,500]
    min_pm_conc = [0,31,61,91,121,251]
    max_pm_conc = [30,60,90,120,250,500]
    min_co_conc = [0,1.1,2.1,10.1,17.1,34.1]
    max_co_conc = [1,2,10,17,34,50]
    min_o3_conc = [0,51,101,169,209,748]
    max_o3_conc = [50,100,168,208,747,1000]
    
    # pm_aqi calculation
    i=0
    pm_aqi = np.full(len(pm_conc),0,dtype=float)
    for pm_val in pm_conc:
        for index,min_pm_val in enumerate(min_pm_conc):
            if pm_val > min_pm_val:
                pm_aqi[i] = ( (max_aqi[index] - min_aqi[index])/(max_pm_conc[index]-min_pm_val) ) * (pm_val - min_pm_val) + min_aqi[index]
            else :
                break
        i = i+1

    # co_aqi calculation
    i=0
    co_aqi = np.full(len(co_conc),0,dtype=float)
    for co_val in co_conc:
        for index,min_co_val in enumerate(min_co_conc):
            if co_val > min_co_val:
                co_aqi[i] = ( (max_aqi[index] - min_aqi[index])/(max_co_conc[index]-min_co_val) ) * (co_val - min_co_val) + min_aqi[index]
            else :
                break
        i = i+1

    # o3_aqi calculation
    i=0
    o3_aqi = np.full(len(o3_conc),0,dtype=float)
    for o3_val in o3_conc:
        for index,min_o3_val in enumerate(min_o3_conc):
            if o3_val > min_o3_val:
                o3_aqi[i] = ( (max_aqi[index] - min_aqi[index])/(max_o3_conc[index]-min_o3_val) ) * (o3_val - min_o3_val) + min_aqi[index]
            else : 
                break
        i = i+1 

    aqi = np.maximum(pm_aqi,np.maximum(co_aqi,o3_aqi))
    return aqi

In [ ]:
aqi = calc_aqi(pm_conc,co_conc,o3_conc)

In [55]:
df = pd.DataFrame({'pm_conc' : pm_conc, 'co_conc' : co_conc,'o3_conc' : o3_conc, 'aqi' : aqi}, index=date_column)

In [56]:
df

,pm_conc,co_conc,o3_conc,aqi
2005-01-01,0.898526,0.046183,25.747099,25.747099
2005-02-01,1.123116,0.047751,25.734440,25.734440
2005-03-01,0.574732,0.045283,24.883772,24.883772
2005-04-01,0.939783,0.050202,25.288557,25.288557
2005-05-01,0.275568,0.049818,25.438364,25.438364
...,...,...,...,...
2023-08-01,0.501248,0.054879,27.092701,27.092701
2023-09-01,0.419525,0.058896,28.963110,28.963110
2023-10-01,0.512932,0.063099,28.883856,28.883856
2023-11-01,0.542980,0.059093,28.340376,28.340376


In [189]:
def forecast_conc(conc_df,exact_date):
    conc_df['datetime'] = conc_df.index.astype('str')
    conc_df = conc_df[['datetime','conc']]
    conc_df.reset_index(drop=True, inplace=True)
    conc_df.columns=['ds','y']
    
    scaler = MinMaxScaler()
    conc_df['y'] = scaler.fit_transform(np.array(conc_df[['y']]))
    conc_df
    
    pro_model = Prophet(interval_width=0.95)
    pro_model.fit(conc_df)

    ref_date = datetime(2024,1,1)
    periods = (exact_date.year - ref_date.year) * 12 + exact_date.month + 1 - ref_date.month 
    future_date = pro_model.make_future_dataframe(periods, freq='MS')
    
    forecast = pro_model.predict(future_date)
    predicted_df = forecast[['ds','yhat']]
    predicted_df.columns=['ds','conc']
    # print(predicted_df.loc[predicted_df.index >= 228,['conc']])
    # np.array(predicted_df.loc[predicted_df.index >= 228,['conc']])
    predicted_df['conc'] = scaler.inverse_transform(predicted_df[['conc']])
    print(predicted_df)
    return predicted_df[predicted_df['ds'] == exact_date.date()].conc

In [190]:
exact_date = datetime(2025,4,1)

pm_conc_df = df.loc[:,['pm_conc']]
pm_conc_df.columns=['conc']

forecast_conc(pm_conc_df, exact_date)

05:45:03 - cmdstanpy - INFO - Chain [1] start processing
05:45:03 - cmdstanpy - INFO - Chain [1] done processing


            ds      conc
0   2005-01-01  0.917397
1   2005-02-01  0.798378
2   2005-03-01  0.663061
3   2005-04-01  0.619188
4   2005-05-01  0.447051
..         ...       ...
239 2024-12-01  1.260943
240 2025-01-01  1.299704
241 2025-02-01  1.181323
242 2025-03-01  1.046581
243 2025-04-01  1.003345

[244 rows x 2 columns]


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13464\3367509547.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df['conc'] = scaler.inverse_transform(predicted_df[['conc']])


Series([], Name: conc, dtype: float64)